In [1]:
import torch
import numpy as np

In [2]:
class Embedder(torch.nn.Module):
    
    def __init__(self, vocab_size, d_model):
        super().__init__()
        self.embed = torch.nn.Embedding(vocab_size, d_model)
        
    def forward(self, x):
        # [123, 0, 23, 5] -> [[..512..], [...512...], ...]
        return self.embed(x)  

In [3]:
import math

class PositionalEncoder(torch.nn.Module):
    
    def __init__(self, d_model, max_seq_len=80):
        super().__init__()
        self.d_model = d_model
        
        # create constant positional encoding matrix
        pe_matrix = torch.zeros(max_seq_len, d_model)
        
        for pos in range(max_seq_len):
            for i in range(0, d_model, 2):
                pe_matrix[pos, i] = math.sin(pos/10000**(2*i/d_model))
                pe_matrix[pos, i+1] = math.cos(pos/10000**(2*i/d_model))
        pe_matrix = pe_matrix.unsqueeze(0)     # Add one dimension for batch size
        self.register_buffer('pe', pe_matrix)  # Register as persistent buffer
        
    def forward(self, x):
        # x is a sentence after embedding with dim (batch, number of words, vector dimension)
        seq_len = x.size()[1]
        x = x + self.pe[:, :seq_len]
        return x

In [4]:
import math
import torch.nn.functional as F

# Given Query, Key, Value, calculate the final weighted value
def scaled_dot_product_attention(q, k, v, mask=None, dropout=None):
    # Shape of q and k are the same, both are (batch_size, seq_len, d_k)
    # Shape of v is (batch_size, seq_len, d_v)
    attention_scores = torch.matmul(q, k.transpose(-2, -1))/math.sqrt(q.shape[-1])  # size (batch_size, seq_len, seq_len)
    
    # Apply mask to scores
    # 
    if mask is not None:
        attention_scores = attention_scores.masked_fill(mask == 0, value=-1e9)
        
    # Softmax along the last dimension
    attention_weights = F.softmax(attention_scores, dim=-1)
    
    if dropout is not None:
        attention_weights = dropout(attention_weights)
        
    output = torch.matmul(attention_weights, v)
    return output

In [5]:
class MultiHeadAttention(torch.nn.Module):
    def __init__(self, n_heads, d_model, dropout=0.1):
        super().__init__()
        
        self.n_heads = n_heads
        self.d_model = d_model
        self.d_k = self.d_v = d_model//n_heads
        
        # self attention linear layers
        # Linear layers for q, k, v vectors generation in different heads
        self.q_linear_layers = []
        self.k_linear_layers = []
        self.v_linear_layers = []
        for i in range(n_heads):
            self.q_linear_layers.append(torch.nn.Linear(d_model, self.d_k))
            self.k_linear_layers.append(torch.nn.Linear(d_model, self.d_k))
            self.v_linear_layers.append(torch.nn.Linear(d_model, self.d_v))
        
        self.dropout = torch.nn.Dropout(dropout)
        self.out = torch.nn.Linear(n_heads*self.d_v, d_model)
        
    def forward(self, q, k, v, mask=None):
        multi_head_attention_outputs = []
        for q_linear, k_linear, v_linear in zip(self.q_linear_layers,
                                                self.k_linear_layers,
                                                self.v_linear_layers):
            new_q = q_linear(q)  # size: (batch_size, seq_len, d_k)
            new_k = k_linear(k)  # size: (batch_size, seq_len, d_k)
            new_v = v_linear(v)  # size (batch_size, seq_len, d_v)
            
            # Scaled Dot-Product attention
            head_v = scaled_dot_product_attention(new_q, new_k, new_v, mask, self.dropout)  # (batch_size, seq_len, d_v)
            multi_head_attention_outputs.append(head_v)
            
        # Concat
        #import pdb; pdb.set_trace()
        concat = torch.cat(multi_head_attention_outputs, -1)  # (batch_size, seq_len, n_heads*d_v)
        
        # Linear layer to recover to original shap
        output = self.out(concat)  # (batch_size, seq_len, d_model)
        
        return output

In [6]:
class FeedForward(torch.nn.Module):
    def __init__(self, d_model, d_ff=2048, dropout=0.1):
        super().__init__()
        
        self.linear_1 = torch.nn.Linear(d_model, d_ff)
        self.dropout = torch.nn.Dropout(dropout)
        self.linear_2 = torch.nn.Linear(d_ff, d_model)
        
    def forward(self, x):
        x = self.dropout(F.relu(self.linear_1(x)))
        x = self.linear_2(x)
        return x

In [7]:
class LayerNorm(torch.nn.Module):
    def __init__(self, d_model, eps=1e-6):
        super().__init__()
        self.d_model = d_model
        self.alpha = torch.nn.Parameter(torch.ones(self.d_model))
        self.beta = torch.nn.Parameter(torch.zeros(self.d_model))
        self.eps = eps
        
    def forward(self, x):
        # x size: (batch_size, seq_len, d_model)
        x_hat = (x - x.mean(dim=-1, keepdim=True))/(x.std(dim=-1, keepdim=True) + self.eps)
        x_tilde = self.alpha*x_hat + self.beta
        return x_tilde

In [8]:
class EncoderLayer(torch.nn.Module):
    def __init__(self, d_model, n_heads, dropout=0.1):
        super().__init__()
        self.d_model = d_model
        self.norm_1 = LayerNorm(d_model)
        self.norm_2 = LayerNorm(d_model)
        self.multi_head_attention = MultiHeadAttention(n_heads, d_model)
        self.feed_forward = FeedForward(d_model)
        self.dropout_1 = torch.nn.Dropout(dropout)
        self.dropout_2 = torch.nn.Dropout(dropout)
        
    def forward(self, x, mask):
        #import pdb; pdb.set_trace()
        x = x + self.dropout_1(self.multi_head_attention(x, x, x, mask))
        x = self.norm_1(x)
        
        x = x + self.dropout_2(self.feed_forward(x))
        x = self.norm_2(x)
        return x

In [9]:
class DecoderLayer(torch.nn.Module):
    def __init__(self, d_model, n_heads, dropout=0.1):
        super().__init__()
        self.norm_1 = LayerNorm(d_model)
        self.norm_2 = LayerNorm(d_model)
        self.norm_3 = LayerNorm(d_model)
        
        self.dropout_1 = torch.nn.Dropout(dropout)
        self.dropout_2 = torch.nn.Dropout(dropout)
        self.dropout_3 = torch.nn.Dropout(dropout)
        
        self.multi_head_attention_1 = MultiHeadAttention(n_heads, d_model)
        self.multi_head_attention_2 = MultiHeadAttention(n_heads, d_model)
        
        self.feed_forward = FeedForward(d_model)
        
    def forward(self, x, encoder_output, src_mask, trg_mask):
        x = self.dropout_1(self.multi_head_attention_1(x, x, x, trg_mask))
        x = x + self.norm_1(x)
        
        x = self.dropout_2(self.multi_head_attention_2(x, encoder_output, encoder_output, src_mask))
        x = x + self.norm_2(x)
        
        x = self.dropout_3(self.feed_forward(x))
        x = x + self.norm_3(x)
        
        return x

In [10]:
import copy

def clone_layer(module, N):
    return torch.nn.ModuleList([copy.deepcopy(module) for i in range(N)])

In [11]:
class Encoder(torch.nn.Module):
    def __init__(self, vocab_size, d_model, N, n_heads):
        super().__init__()
        self.embed = Embedder(vocab_size, d_model)
        self.pe = PositionalEncoder(d_model)
        self.encoder_layers = clone_layer(EncoderLayer(d_model, n_heads), N)
        self.norm = LayerNorm(d_model)
        
    def forward(self, src, mask):
        x = self.embed(src)
        x = self.pe(x)
        for encoder in self.encoder_layers:
            x = encoder(x, mask)
        return self.norm(x)

In [12]:
class Decoder(torch.nn.Module):
    def __init__(self, vocab_size, d_model, N, n_heads):
        super().__init__()
        self.embed = Embedder(vocab_size, d_model)
        self.pe = PositionalEncoder(d_model)
        self.decoder_layers = clone_layer(DecoderLayer(d_model, n_heads), N)
        self.norm = LayerNorm(d_model)
        
    def forward(self, trg, encoder_output, src_mask, trg_mask):
        x = self.embed(trg)
        x = self.pe(x)
        for decoder in self.decoder_layers:
            x = decoder(x, encoder_output, src_mask, trg_mask)
        return self.norm(x)

In [13]:
class Transformer(torch.nn.Module):
    def __init__(self, src_vocab_size, trg_vocab_size, d_model, N, n_heads):
        super().__init__()
        self.encoder = Encoder(src_vocab_size, d_model, N, n_heads)
        self.decoder = Decoder(trg_vocab_size, d_model, N, n_heads)
        self.linear = torch.nn.Linear(d_model, trg_vocab_size)
        
    def forward(self, src, trg, src_mask, trg_mask):
        encoder_output = self.encoder(src, src_mask)
        decoder_output = self.decoder(trg, encoder_output, src_mask, trg_mask)
        output = self.linear(decoder_output)
        return output

In [2]:
import spacy
from tqdm.auto import tqdm
from torchtext import data
nlp = spacy.load('pt_core_news_sm')

tokenizer = lambda sentence: [tok.text for tok in nlp.tokenizer(sentence) if tok.text != " "]

eng = open('data/english.txt', 'r', encoding = 'utf8').readlines()
fr = open('data/french.txt', 'r', encoding = 'utf8').readlines()

engnew = open('data/english1.txt', 'w', encoding = 'utf8')
frnew = open('data/french1.txt', 'w', encoding = 'utf8')

for e, f in tqdm(zip(eng, fr)):
    e, f = tokenizer(e)[:-1], ['@'] + tokenizer(f)[:-1] + ['#']
    for i in range(1, len(f)):
#         print(e, f[:i], [f[i]])
        engnew.write(' '.join(e))
        engnew.write('\n')
        frnew.write(' '.join(f[:i+1]))
        frnew.write('\n')

engnew.close()
frnew.close()

0it [00:00, ?it/s]

In [22]:
engnew.close()
frnew.close()
ys.close()

In [5]:
import spacy
from torchtext import data
nlp = spacy.load('pt_core_news_sm')

tokenizer = lambda sentence: [tok.text for tok in nlp.tokenizer(sentence) if tok.text != " "]
SRC = data.Field(lower=True, tokenize=tokenizer)
TRG = data.Field(lower=True, tokenize=tokenizer)
Y = data.Field(lower=True, tokenize=tokenizer)
src_data = open('data/english1.txt', 'r', encoding = 'utf8')
trg_data = open('data/french1.txt', 'r', encoding = 'utf8')
raw_data = {'src': [line for line in src_data], 'trg': [line for line in trg_data]}
import pandas as pd
df = pd.DataFrame(raw_data, columns=['src', 'trg'])
df.tail(50)

,src,trg
1343731,If someone who doesn't know your background sa...,@ Si quelqu'un qui ne connaît pas vos antécéde...
1343732,If someone who doesn't know your background sa...,@ Si quelqu'un qui ne connaît pas vos antécéde...
1343733,If someone who doesn't know your background sa...,@ Si quelqu'un qui ne connaît pas vos antécéde...
1343734,If someone who doesn't know your background sa...,@ Si quelqu'un qui ne connaît pas vos antécéde...
1343735,If someone who doesn't know your background sa...,@ Si quelqu'un qui ne connaît pas vos antécéde...
1343736,If someone who doesn't know your background sa...,@ Si quelqu'un qui ne connaît pas vos antécéde...
1343737,If someone who doesn't know your background sa...,@ Si quelqu'un qui ne connaît pas vos antécéde...
1343738,If someone who doesn't know your background sa...,@ Si quelqu'un qui ne connaît pas vos antécéde...
1343739,If someone who doesn't know your background sa...,@ Si quelqu'un qui ne connaît pas vos antécéde...
1343740,If someone who doesn't know your background sa...,@ Si quelqu'un qui ne connaît pas vos antécéde...


In [18]:
import spacy
from torchtext import data
nlp = spacy.load('pt_core_news_sm')

tokenizer = lambda sentence: [tok.text for tok in nlp.tokenizer(sentence) if tok.text != " "]
SRC = data.Field(lower=True, tokenize=tokenizer)
TRG = data.Field(lower=True, tokenize=tokenizer, init_token = '<sos>', eos_token = '<eos>')

src_data = open('data/english.txt', 'r', encoding = 'utf8')
trg_data = open('data/french.txt', 'r', encoding = 'utf8')
raw_data = {'src': [line for line in src_data], 'trg': [line for line in trg_data]}

import pandas as pd
df = pd.DataFrame(raw_data, columns=['src', 'trg'])
df.head(50)

,src,trg
0,Go.\n,Va !\n
1,Run!\n,Cours !\n
2,Run!\n,Courez !\n
3,Fire!\n,Au feu !\n
4,Help!\n,À l'aide !\n
5,Jump.\n,Saute.\n
6,Stop!\n,Ça suffit !\n
7,Stop!\n,Stop !\n
8,Stop!\n,Arrête-toi !\n
9,Wait!\n,Attends !\n


In [19]:
df.to_csv('en_to_fr.csv', index=False)

data_fields = [('src', SRC), ('trg', TRG)]
train_set = data.TabularDataset('./en_to_fr.csv', format='csv', fields=data_fields)
SRC.build_vocab(train_set)
print(len(SRC.vocab))

TRG.build_vocab(train_set)
print(len(TRG.vocab))

train_set

14115
28354


In [20]:
[SRC.vocab.stoi[tok] for tok in tokenizer("A Run carbon footprint is the amount of carbon")]

[0, 0, 3513, 5974, 9, 6, 2025, 15, 3513]

In [21]:
# set some parameters
d_model = 512
n_heads = 8
N = 6
src_vocab_size = len(SRC.vocab)
trg_vocab_size = len(TRG.vocab)
model = Transformer(src_vocab_size, trg_vocab_size, d_model, N, n_heads)
for p in model.parameters():
    if p.dim() > 1:
        torch.nn.init.xavier_uniform_(p)

In [22]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)
train_iter = data.Iterator(train_set, batch_size=1, sort_key=lambda x: (len(x.src), len(x.trg)), shuffle=True, train=True)
import numpy as np
from tqdm.auto import tqdm

def create_mask(src_input, trg_input):
    # Source input mask
    pad = SRC.vocab.stoi['<pad>']
    src_mask = (src_input != pad).unsqueeze(1)
    
    # Target input mask
    trg_mask = (trg_input != pad).unsqueeze(1)
    
    seq_len = trg_input.size(1)
    nopeak_mask = np.tril(np.ones((1, seq_len, seq_len)), k=0).astype('uint8')
    nopeak_mask = torch.from_numpy(nopeak_mask) != 0
    trg_mask = trg_mask & nopeak_mask
    
    return src_mask, trg_mask

import time

def train_model(n_epochs, output_interval=100):
    model.train()
    start = time.time()
    
    for epoch in range(n_epochs):
        
        total_loss = 0
        for i, batch in enumerate(train_iter):
            
            src_input = batch.src.transpose(0, 1)  # size (batch_size, seq_len)
            trg = batch.trg.transpose(0, 1)  # size (batch_size, seq_len)
            
            trg_input = trg[:, :-1]
            ys = trg[:, 1:].contiguous().view(-1)
            # create src & trg masks
            src_mask, trg_mask = create_mask(src_input, trg_input)
            preds = model(src_input, trg_input, src_mask, trg_mask)
            
            optimizer.zero_grad()
            loss = F.cross_entropy(preds.view(-1, preds.size(-1)), ys, ignore_index=1)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.data

            if (i + 1) % output_interval == 0:
                avg_loss = total_loss/output_interval
                print('time = {}, epoch = {}, iter = {}, loss = {}'.format((time.time() - start)/60,
                                                                           epoch + 1,
                                                                           i + 1,
                                                                           avg_loss))
                total_loss = 0
                start = time.time()

In [27]:
for i, batch in enumerate(train_iter):
    src_input = batch.src.transpose(0,1)
    trg = batch.trg.transpose(0,1) # size (batch_size, seq_len)
    
    trg_input = trg[:, :-1]
    ys = trg[:, 1:].contiguous().view(-1)
    for s, t in zip(src_input, trg_input):
        print([SRC.vocab.itos[tok] for tok in s])
        print([TRG.vocab.itos[tok] for tok in t])
        print([TRG.vocab.itos[tok] for tok in ys])
        print('-'*100)
    print('='*100)
    
    src_mask, trg_mask = create_mask(src_input, trg_input)
    print(trg_mask)
    preds = model(src_input, trg_input, src_mask, trg_mask)
    preds = F.softmax(preds, dim = -1)
    _, out = preds.data.topk(1)
    print([TRG.vocab.itos[tok] for tok in out[0]])
    break

NameError: name 'train_iter' is not defined

In [25]:
def t(src, trg):
    src_input = torch.tensor([[SRC.vocab.stoi[tok] for tok in src]])
    trg = torch.tensor([[TRG.vocab.stoi[tok] for tok in trg]])
    
    trg_input = trg[:, :-1]
    
    src_mask, trg_mask = create_mask(src_input, trg_input)
    print(src_mask, trg_mask)
    preds = model(src_input, trg_input, src_mask, trg_mask)
    print(preds)
    preds = F.softmax(preds, dim = -1)
    _, out = preds.data.topk(1)
    print([TRG.vocab.itos[tok] for tok in out[0]])

src = ['have', 'you', 'ever', 'seen', 'a', 'tiger', 'around', 'here', '?'] 
trg = ['<sos>', 'avez-vous', 'déjà', 'vu', 'un', 'tigre', 'dans', 'les', 'environs', '?', '<eos>']

def translate(src, max_len = 50):
    src_input = torch.tensor([[SRC.vocab.stoi[tok] for tok in src]])
    trg_input = torch.tensor([[TRG.vocab.stoi['<sos>']]])
    
    for _ in range(max_len):
        src_mask, trg_mask = create_mask(src_input, trg_input)
        preds = model(src_input, trg_input, src_mask, trg_mask)
        preds = F.softmax(preds, dim = -1)
        _, out = preds.data.topk(1)
        trg_input = torch.cat((trg_input, torch.tensor([[out[0][-1]]])))
        if out[0][-1][0] == torch.tensor(TRG.vocab.stoi['<eos>']):
            break
    return ' '.join([TRG.vocab.itos[tok] for tok in trg_input])

translate(src)

'<sos> que pourquoi pourquoi comment comment pourquoi pourquoi pourquoi comment pourquoi que pourquoi pourquoi pourquoi comment pourquoi comment comment pourquoi comment que que comment pourquoi comment pourquoi pourquoi pourquoi pourquoi comment pourquoi comment que que pourquoi comment comment pourquoi pourquoi comment pourquoi comment comment pourquoi pourquoi comment pourquoi pourquoi que pourquoi'

In [ ]:
train_model(3, output_interval=1)

time = 0.016568076610565186, epoch = 1, iter = 1, loss = 6.364777565002441
time = 0.01610981225967407, epoch = 1, iter = 2, loss = 7.785104274749756
time = 0.01474459966023763, epoch = 1, iter = 3, loss = 6.641254425048828
time = 0.015484038988749187, epoch = 1, iter = 4, loss = 8.166705131530762
time = 0.015028854211171469, epoch = 1, iter = 5, loss = 6.028402805328369
time = 0.015239071846008301, epoch = 1, iter = 6, loss = 5.850545883178711
time = 0.01653244098027547, epoch = 1, iter = 7, loss = 7.139897346496582
time = 0.020486164093017577, epoch = 1, iter = 8, loss = 6.179225921630859
time = 0.021252556641896566, epoch = 1, iter = 9, loss = 7.808310508728027
time = 0.01576830546061198, epoch = 1, iter = 10, loss = 7.755741119384766
time = 0.015081632137298583, epoch = 1, iter = 11, loss = 7.039430618286133
time = 0.014724949995676676, epoch = 1, iter = 12, loss = 6.282425880432129
time = 0.015323448181152343, epoch = 1, iter = 13, loss = 6.6631879806518555
time = 0.016498037179311

time = 0.014906509717305502, epoch = 1, iter = 110, loss = 7.603344917297363
time = 0.01490335464477539, epoch = 1, iter = 111, loss = 7.069977760314941
time = 0.015535569190979004, epoch = 1, iter = 112, loss = 6.010895252227783
time = 0.015240983168284098, epoch = 1, iter = 113, loss = 6.997994899749756
time = 0.015251576900482178, epoch = 1, iter = 114, loss = 5.298720359802246
time = 0.014977351824442545, epoch = 1, iter = 115, loss = 7.077021598815918
time = 0.015481801827748616, epoch = 1, iter = 116, loss = 6.601924896240234
time = 0.0149750550587972, epoch = 1, iter = 117, loss = 6.3649797439575195
time = 0.014788695176442464, epoch = 1, iter = 118, loss = 7.241326808929443
time = 0.015041780471801759, epoch = 1, iter = 119, loss = 6.903527736663818
time = 0.0151214599609375, epoch = 1, iter = 120, loss = 7.8203277587890625
time = 0.015495137373606364, epoch = 1, iter = 121, loss = 5.683116912841797
time = 0.014774616559346516, epoch = 1, iter = 122, loss = 6.963935852050781
ti

time = 0.016067111492156984, epoch = 1, iter = 217, loss = 7.981557369232178
time = 0.015085152784983317, epoch = 1, iter = 218, loss = 5.71095609664917
time = 0.015858805179595946, epoch = 1, iter = 219, loss = 5.640440464019775
time = 0.015534520149230957, epoch = 1, iter = 220, loss = 5.59948205947876
time = 0.015269529819488526, epoch = 1, iter = 221, loss = 6.7716827392578125
time = 0.015584909915924072, epoch = 1, iter = 222, loss = 7.336301326751709
time = 0.014829460779825847, epoch = 1, iter = 223, loss = 4.580948352813721
time = 0.014569528897603353, epoch = 1, iter = 224, loss = 7.896090030670166
time = 0.01504906415939331, epoch = 1, iter = 225, loss = 6.481283187866211
time = 0.015131346384684245, epoch = 1, iter = 226, loss = 5.767949104309082
time = 0.015105827649434408, epoch = 1, iter = 227, loss = 5.8119330406188965
time = 0.015232284863789877, epoch = 1, iter = 228, loss = 7.1722846031188965
time = 0.015360653400421143, epoch = 1, iter = 229, loss = 6.303107738494873

time = 0.01586843729019165, epoch = 1, iter = 324, loss = 6.362663745880127
time = 0.015098853905995687, epoch = 1, iter = 325, loss = 5.912585258483887
time = 0.015066742897033691, epoch = 1, iter = 326, loss = 6.430394649505615
time = 0.014976723988850912, epoch = 1, iter = 327, loss = 6.621278285980225
time = 0.015549282232920328, epoch = 1, iter = 328, loss = 5.325039386749268
time = 0.01523664395014445, epoch = 1, iter = 329, loss = 6.074916839599609
time = 0.015516308943430583, epoch = 1, iter = 330, loss = 5.36292028427124
time = 0.015260740121205648, epoch = 1, iter = 331, loss = 6.035889625549316
time = 0.01561439037322998, epoch = 1, iter = 332, loss = 6.732856273651123
time = 0.016059676806132, epoch = 1, iter = 333, loss = 6.188155174255371
time = 0.016783646742502847, epoch = 1, iter = 334, loss = 6.16618013381958
time = 0.017112255096435547, epoch = 1, iter = 335, loss = 7.148543357849121
time = 0.01581275463104248, epoch = 1, iter = 336, loss = 7.1462931632995605
time = 

time = 0.01578883727391561, epoch = 1, iter = 431, loss = 6.868432521820068
time = 0.015361074606577556, epoch = 1, iter = 432, loss = 6.299765110015869
time = 0.015064589182535807, epoch = 1, iter = 433, loss = 6.608158111572266
time = 0.015561517079671223, epoch = 1, iter = 434, loss = 6.641746520996094
time = 0.015254473686218262, epoch = 1, iter = 435, loss = 6.348840713500977
time = 0.014995201428731283, epoch = 1, iter = 436, loss = 5.750905990600586
time = 0.014969058831532796, epoch = 1, iter = 437, loss = 6.064032077789307
time = 0.015250396728515626, epoch = 1, iter = 438, loss = 5.910512447357178
time = 0.014989511171976725, epoch = 1, iter = 439, loss = 5.739227294921875
time = 0.01481003761291504, epoch = 1, iter = 440, loss = 6.02882194519043
time = 0.015041184425354005, epoch = 1, iter = 441, loss = 6.1530351638793945
time = 0.015590973695119222, epoch = 1, iter = 442, loss = 7.773530960083008
time = 0.015501093864440919, epoch = 1, iter = 443, loss = 7.6615400314331055


...

time = 0.019540786743164062, epoch = 1, iter = 3722, loss = 6.125877857208252
time = 0.018256521224975585, epoch = 1, iter = 3723, loss = 6.465667247772217
time = 0.017397181193033854, epoch = 1, iter = 3724, loss = 5.7624616622924805
time = 0.01794694662094116, epoch = 1, iter = 3725, loss = 6.260304927825928
time = 0.017326295375823975, epoch = 1, iter = 3726, loss = 7.118714809417725
time = 0.017923760414123534, epoch = 1, iter = 3727, loss = 5.532364368438721
time = 0.017920068899790444, epoch = 1, iter = 3728, loss = 5.516927242279053
time = 0.017340493202209473, epoch = 1, iter = 3729, loss = 6.022172451019287
time = 0.017966564496358237, epoch = 1, iter = 3730, loss = 6.108994483947754
time = 0.017413302262624105, epoch = 1, iter = 3731, loss = 6.992203235626221
time = 0.017863746484120688, epoch = 1, iter = 3732, loss = 6.663990020751953
time = 0.017896461486816406, epoch = 1, iter = 3733, loss = 6.340667724609375
time = 0.017897661526997885, epoch = 1, iter = 3734, loss = 7.88

time = 0.017994999885559082, epoch = 1, iter = 3828, loss = 7.852565288543701
time = 0.017639307181040446, epoch = 1, iter = 3829, loss = 6.51274299621582
time = 0.017918145656585692, epoch = 1, iter = 3830, loss = 7.572894096374512
time = 0.018161861101786296, epoch = 1, iter = 3831, loss = 5.251627445220947
time = 0.017875981330871583, epoch = 1, iter = 3832, loss = 5.853228569030762
time = 0.017709954579671224, epoch = 1, iter = 3833, loss = 5.271236896514893
time = 0.01770333449045817, epoch = 1, iter = 3834, loss = 5.390597343444824
time = 0.018104310830434164, epoch = 1, iter = 3835, loss = 7.429394245147705
time = 0.017704566319783527, epoch = 1, iter = 3836, loss = 7.374758243560791
time = 0.018439829349517822, epoch = 1, iter = 3837, loss = 6.714697360992432
time = 0.017694687843322753, epoch = 1, iter = 3838, loss = 6.491124629974365
time = 0.0179482897122701, epoch = 1, iter = 3839, loss = 5.55996036529541
time = 0.017619967460632324, epoch = 1, iter = 3840, loss = 5.0814638

time = 0.018229846159617105, epoch = 1, iter = 3934, loss = 5.171271324157715
time = 0.01773087978363037, epoch = 1, iter = 3935, loss = 7.748638153076172
time = 0.01783454418182373, epoch = 1, iter = 3936, loss = 6.59873628616333
time = 0.018166104952494305, epoch = 1, iter = 3937, loss = 6.829709053039551
time = 0.017700032393137614, epoch = 1, iter = 3938, loss = 4.5057053565979
time = 0.018075899283091227, epoch = 1, iter = 3939, loss = 5.9138569831848145
time = 0.017754840850830077, epoch = 1, iter = 3940, loss = 6.44536018371582
time = 0.0181752880414327, epoch = 1, iter = 3941, loss = 5.432521343231201
time = 0.018106762568155924, epoch = 1, iter = 3942, loss = 6.073098182678223
time = 0.017907615502675375, epoch = 1, iter = 3943, loss = 6.361081123352051
time = 0.01789497137069702, epoch = 1, iter = 3944, loss = 6.267362117767334
time = 0.018133715788523356, epoch = 1, iter = 3945, loss = 5.781929016113281
time = 0.017968432108561198, epoch = 1, iter = 3946, loss = 6.5528483390

time = 0.018216880162556966, epoch = 1, iter = 4040, loss = 5.879962921142578
time = 0.017965126037597656, epoch = 1, iter = 4041, loss = 6.761811256408691
time = 0.018205785751342775, epoch = 1, iter = 4042, loss = 4.909966945648193
time = 0.018093395233154296, epoch = 1, iter = 4043, loss = 6.80560302734375
time = 0.01765897274017334, epoch = 1, iter = 4044, loss = 5.163455009460449
time = 0.01792608102162679, epoch = 1, iter = 4045, loss = 6.3885345458984375
time = 0.01835795243581136, epoch = 1, iter = 4046, loss = 8.101806640625
time = 0.01819608211517334, epoch = 1, iter = 4047, loss = 6.678640365600586
time = 0.01842950185139974, epoch = 1, iter = 4048, loss = 5.888946533203125
time = 0.017902863025665284, epoch = 1, iter = 4049, loss = 6.641042232513428
time = 0.018268016974131267, epoch = 1, iter = 4050, loss = 5.31168270111084
time = 0.018159870306650797, epoch = 1, iter = 4051, loss = 6.913180351257324
time = 0.01815632184346517, epoch = 1, iter = 4052, loss = 6.456026554107

time = 0.019922180970509847, epoch = 1, iter = 4146, loss = 6.323391437530518
time = 0.020954608917236328, epoch = 1, iter = 4147, loss = 5.260069847106934
time = 0.018714559078216553, epoch = 1, iter = 4148, loss = 6.190357208251953
time = 0.018378170331319173, epoch = 1, iter = 4149, loss = 5.725307464599609
time = 0.01788225571314494, epoch = 1, iter = 4150, loss = 7.157690525054932
time = 0.01794639031092326, epoch = 1, iter = 4151, loss = 6.3947296142578125
time = 0.018736358483632407, epoch = 1, iter = 4152, loss = 5.629854679107666
time = 0.017957464853922526, epoch = 1, iter = 4153, loss = 5.401864528656006
time = 0.018422508239746095, epoch = 1, iter = 4154, loss = 6.431536674499512
time = 0.01762666702270508, epoch = 1, iter = 4155, loss = 5.739795207977295
time = 0.01818012793858846, epoch = 1, iter = 4156, loss = 5.713539123535156
time = 0.018160812060038247, epoch = 1, iter = 4157, loss = 6.129301071166992
time = 0.018560985724131267, epoch = 1, iter = 4158, loss = 5.56935

In [40]:
PATH = "Trained_model_NEW"
torch.save(model, PATH)

In [24]:
model = torch.load("Trained_model_NEW")

In [52]:
def translate(model, src, max_len = 80, custom_string=False):    
    model.eval()
    if custom_string == True:
            src = tokenizer(src)
            src = torch.LongTensor([[SRC.vocab.stoi[tok] for tok in src]])
    
#     pad = SRC.vocab.stoi['<pad>']
#     src_mask = (src != pad).unsqueeze(1)
#     e_outputs = model.encoder(src, src_mask)
    
    outputs = torch.zeros(max_len).type_as(src.data)
    outputs[0] = torch.LongTensor([TRG.vocab.stoi['<sos>']])
    for i in range(1, max_len):    

#             trg_mask = np.tril(np.ones((1, i, i)), k = 0).astype('uint8')
#             trg_mask = torch.from_numpy(trg_mask) != 0
#             print(outputs[:i], e_outputs)

            src_mask, trg_mask = create_mask(src, outputs[:i].unsqueeze(0))
            print(outputs[:i].unsqueeze(0))
            out = model(src, outputs[:i].unsqueeze(0), src_mask, trg_mask)
            out = F.softmax(out, dim=-1)
            val, ix = out[:, -1].data.topk(1)

            outputs[i] = ix[0][0]
            if ix[0][0] == TRG.vocab.stoi['<eos>']:
                break
    return ' '.join([TRG.vocab.itos[ix] for ix in outputs[:i]])
                               
translate(model, "A carbon footprint is the amount of carbon", custom_string = True)

tensor([[2]])
tensor([[ 2, 16]])
tensor([[ 2, 16, 16]])
tensor([[ 2, 16, 16, 16]])
tensor([[ 2, 16, 16, 16, 16]])
tensor([[ 2, 16, 16, 16, 16, 16]])
tensor([[ 2, 16, 16, 16, 16, 16, 16]])
tensor([[ 2, 16, 16, 16, 16, 16, 16, 16]])
tensor([[ 2, 16, 16, 16, 16, 16, 16, 16, 16]])
tensor([[ 2, 16, 16, 16, 16, 16, 16, 16, 16, 16]])
tensor([[ 2, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16]])
tensor([[ 2, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16]])
tensor([[ 2, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16]])
tensor([[ 2, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16]])
tensor([[ 2, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16]])
tensor([[ 2, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16]])
tensor([[ 2, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16]])
tensor([[ 2, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16]])
tensor([[ 2, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
         16]])
tensor([[ 2, 16, 16, 16

tensor([[ 2, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
         16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
         16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
         16, 16, 16, 16, 16, 16, 16]])
tensor([[ 2, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
         16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
         16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
         16, 16, 16, 16, 16, 16, 16, 16]])
tensor([[ 2, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
         16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
         16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
         16, 16, 16, 16, 16, 16, 16, 16, 16]])
tensor([[ 2, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
         16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 

'<sos> tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom tom'